In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tinyyolov2NoBN import TinyYoloV2NoBN 

In [2]:
model = TinyYoloV2NoBN(num_classes=1)
sd = torch.load("fusedyolov2_0717.pt")
model.load_state_dict(sd, strict=False)
#print(sd.keys())
model.eval()

TinyYoloV2NoBN(
  (pad): ReflectionPad2d((0, 1, 0, 1))
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv7): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv8): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv9): Conv2d(1024, 30, kernel_size=(1, 1), stride=(1, 1))
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

TinyYoloV2NoBN(
  (pad): ReflectionPad2d((0, 1, 0, 1))
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv7): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv8): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv9): Conv2d(1024, 30, kernel_size=(1, 1), stride=(1, 1))
)

In [4]:
#import torch.nn.utils.prune as prune
#parameters_to_prune = (
#    (model.conv1, 'weight'),
#    (model.conv2, 'weight'),
#    (model.conv3, 'weight'),
#    (model.conv4, 'weight'),
#    (model.conv5, 'weight'),
#    (model.conv6, 'weight'),
    (model.conv7, 'weight'),
    (model.conv8, 'weight'),
    (model.conv9, 'weight'),
)

#for module, name in parameters_to_prune:
    prune.ln_structured(module, name=name, amount=0.2, n=1, dim=0)  # 20% pruning


In [4]:
#print(model.conv1.weight)
print(model)

TinyYoloV2NoBN(
  (pad): ReflectionPad2d((0, 1, 0, 1))
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv7): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv8): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv9): Conv2d(1024, 30, kernel_size=(1, 1), stride=(1, 1))
)


In [4]:
#prune.ln_structured(model.conv1, name='weight', amount=0.2, n=1, dim=0)

# Print the first layer's weights after pruning
#print("First layer weights after pruning:")
#print(model.conv1.weight)

# Remove pruning reparametrizations to make the model dense
#prune.remove(model.conv1, 'weight')

# Print the first layer's weights after densifying
#print("First layer weights after densifying:")
#print(model.conv1.weight)

import torch.nn.utils.prune as prune

#print("Original model:")
#print(model.conv1.weight)

def pruning(model, amount=0.2, n=1, dim=0):
    for module_name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=n, dim=dim)  # 20% pruning      

#pruning(model)
#print("First layer weights after pruning:")
#print(model.conv1.weight)


def remove_pruning(model):
    for module_name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.remove(module, 'weight')
            
#remove_pruning(model)
#print("First layer weights after densifying:")
#print(model.conv1.weight)

In [5]:
#torch.save(model.state_dict(), 'densified_tinyyolov2NoBN.pt')

In [5]:
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss, _ = criterion(outputs, targets)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

In [6]:
import torch.optim as optim
from utils.loss import YoloLoss
import tqdm

def prune_and_retrain(model, train_loader, val_loader, num_iterations, pruning_amount, epochs_per_iteration):
    criterion = YoloLoss(anchors=model.anchors)
    optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr=0.001)
    
    for iteration in range(num_iterations):
        print(f"Pruning iteration {iteration+1}/{num_iterations}")
        
        pruning(model, amount=pruning_amount)
        
        #print(f"Pruned.")
        model.train()
        #print(f"Trained.")
        for epoch in range(epochs_per_iteration):
            running_loss = 0.0
            print(f"Begin Epoch {epoch}")
            for idx, (images, targets) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
                images, targets = images.to(device), targets.to(device)
                print(f"Image, Target loaded.")
                optimizer.zero_grad()
                outputs = model(images, yolo=False)
                print(f"Get Outputs.")
                loss, _ = criterion(outputs, targets)
                print(f"Get Loss.")
                loss.backward()
                print(f"Backwarded.")
                optimizer.step()
                running_loss += loss.item()
            print(f"Epoch [{epoch+1}/{epochs_per_iteration}], Loss: {running_loss/len(train_loader)}")
            
        evaluate_model(model, val_loader, criterion)
        
        remove_pruning(model)
        

In [7]:
import torch

# A subset of VOCDataLoader just for one class (person) (0)
from utils.dataloader import VOCDataLoaderPerson

loader = VOCDataLoaderPerson(train=True, batch_size=1, shuffle=True)
loader_test = VOCDataLoaderPerson(train=False, batch_size=1)

prune_and_retrain(model, loader, loader_test, 5, 0.2, 5)

Pruning iteration 1/5
Begin Epoch 0


  0%|          | 0/2142 [00:00<?, ?it/s]

Image, Target loaded.


  0%|          | 0/2142 [00:30<?, ?it/s]


KeyboardInterrupt: 

In [6]:
import torch

# A subset of VOCDataLoader just for one class (person) (0)
from utils.dataloader import VOCDataLoaderPerson

loader = VOCDataLoaderPerson(train=True, batch_size=1, shuffle=True)
loader_test = VOCDataLoaderPerson(train=False, batch_size=1)

from tinyyolov2NoBN import TinyYoloV2NoBN
from utils.loss import YoloLoss
import tqdm

# We define a tinyyolo network with only two possible classes
net = TinyYoloV2NoBN(num_classes=1)
sd = torch.load("fusedyolov2_0717.pt", map_location=device)

#We load all parameters from the pretrained dict except for the last layer
#net.load_state_dict({k: v for k, v in sd.items() if not '9' in k}, strict=False)
net.load_state_dict(sd, strict=False)
net.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Definition of the loss
criterion = YoloLoss(anchors=net.anchors)

#We only train the last layer (conv9)
#for key, param in net.named_parameters():
#    if any(x in key for x in ['1', '2', '3', '4', '5', '6', '7']):
#        param.requires_grad = False

optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, net.parameters()), lr=0.001)



In [ ]:
from utils.ap import precision_recall_levels, ap, display_roc
from utils.yolo import nms, filter_boxes

NUM_TEST_SAMPLES = 30
NUM_EPOCHS = 20
test_AP = []
num_iterations = 20
ratio = 0.1

for iteration in range(num_iterations):
    print(f"Pruning iteration {iteration+1}/{num_iterations}")
        
    pruning(net, amount=ratio)
        
    for epoch in range(NUM_EPOCHS):
        if epoch >= 0:
            net.train()
            net.to(device)
            print("Training started.")
            for idx, (input, target) in tqdm.tqdm(enumerate(loader), total=len(loader)):
                input, target = input.to(device), target.to(device)
                optimizer.zero_grad()
                #Yolo head is implemented in the loss for training, therefore yolo=False
                output = net(input, yolo=False)
                loss, _ = criterion(output, target)
                loss.backward()
                optimizer.step()

            
        test_precision = []
        test_recall = []
        net.eval()
        print("Validation started.")
        with torch.no_grad():
            for idx, (input, target) in tqdm.tqdm(enumerate(loader_test), total=NUM_TEST_SAMPLES):
                input, target = input.to(device), target.to(device)
                output = net(input, yolo=True)
        
                #The right threshold values can be adjusted for the target application
                output = filter_boxes(output, 0.0)
                output = nms(output, 0.5)
        
                precision, recall = precision_recall_levels(target[0], output[0])
                test_precision.append(precision)
                test_recall.append(recall)
                if idx == NUM_TEST_SAMPLES:
                    break
                
        #Calculation of average precision with collected samples
        test_AP.append(ap(test_precision, test_recall))
        print('average precision', test_AP)

        #plot ROC
        display_roc(test_precision, test_recall)
        
    remove_pruning(net)
    state_dict = net.state_dict()
        
torch.save(state_dict, 'zhz_sr_pruned.pt')

Pruning iteration 1/20
Training started.


  0%|          | 0/2142 [00:00<?, ?it/s]

torch.Size([1, 30, 10, 10])
